In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import pandas as pd

# --- Import your scoring classes here ---
from aggregate_scoring import (
     CommunityTransportationOptions,
     DesirableUndesirableActivities,
     HousingNeedsCharacteristics,
     QualityEducation,
     StableCommunities
)

# --- Define Grid Parameters ---
lon_min, lon_max = -84.911059, -83.799104
lat_min, lat_max = 33.256500, 34.412590
step = 0.01

lats = np.arange(lat_min, lat_max, step)
lons = np.arange(lon_min, lon_max, step)

records = []

# --- Loop over lat/lon grid ---
for lat in tqdm(lats, desc="Latitude rows"):
    for lon in lons:
        try:
            kwargs = {
            # --- CommunityTransportationOptions ---
            "transit_df": pd.read_csv("../../data/raw/scoring_indicators/community_trans_options_sites/georgia_transit_locations_with_hub.csv"),

            # --- DesirableUndesirableActivities ---
            "rural_gdf_unary_union": gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054/USDA_Rural_Housing_by_Tract.shp").to_crs("EPSG:4326").unary_union,
            "desirable_csv": pd.read_csv("../../data/processed/scoring_indicators/desirable_activities_google_places_v3.csv"),
            "grocery_csv": pd.read_csv("../../data/processed/scoring_indicators/desirable_activities_google_places_v3.csv"),
            "usda_csv": pd.read_csv("../../data/raw/scoring_indicators/food_access_research_atlas.csv", dtype={'CensusTract': str}),
            "tract_shapefile": gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp"),
            "undesirable_csv": pd.read_csv("../../data/processed/scoring_indicators/undesirable_hsi_tri_cdr_rcra_frs_google_places.csv"),

            # --- QualityEducation ---
            "school_df": pd.read_csv("../../data/processed/scoring_indicators/quality_education/Option_C_Scores_Eligibility_with_BTO.csv"),
            "combined_gdf": gpd.read_file("../../data/processed/scoring_indicators/quality_education/merged1.geojson", crs="EPSG:4326"),
            "state_avg_by_year": {
                "elementary": {
                    2018: 77.8,
                    2019: 79.9
                },
                "middle": {
                    2018: 76.2,
                    2019: 77
                },
                "high": {
                    2018: 75.3,
                    2019: 78.8
                }
            },

            # --- StableCommunities ---
            "indicators_df": pd.read_csv("../../data/processed/scoring_indicators/stable_communities_2024_processed_v3.csv"),

            # --- HousingNeedsCharacteristics ---
            "census_tract_data": pd.read_csv("../../data/processed/scoring_indicators/housing_needs/merged_housing_data.csv"),
            #"revitalization_score": 4,
            "in_qct": False  # Required for housing need eligibility
            }

            ct_score = CommunityTransportationOptions(lat, lon, **kwargs).calculate_score()
            dua_score = DesirableUndesirableActivities(lat, lon, **kwargs).calculate_score()
            sc_score = StableCommunities(lat, lon, **kwargs).calculate_score()
            hnc_score = HousingNeedsCharacteristics(lat, lon, stable_community_score=sc_score, **kwargs).calculate_score()
            qe_score = QualityEducation(lat, lon, **kwargs).calculate_score()

            total = ct_score + dua_score + hnc_score + qe_score + sc_score

            records.append({
                "lat": lat,
                "lon": lon,
                "community_transportation_options_score": ct_score,
                "desirable_undesirable_activities_score": dua_score,
                "housing_need_characteristics_score": hnc_score,
                "quality_education_areas_score": qe_score,
                "stable_communities_score": sc_score,
                "total_score": total,
                "geometry": Point(lon, lat)
            })

        except Exception as e:
            print(f"Error at (lat={lat:.4f}, lon={lon:.4f}): {e}")

# --- Export GeoJSON ---
gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")
out_path = "../../data/maps/total_location_score/final_scoring_dataset.geojson"
gdf.to_file(out_path, driver="GeoJSON")


In [ ]:
import geopandas as gpd
gdf = gpd.read_file("../../data/maps/total_location_score/LIHTC-Project_data_map_final_scoring_by_category_multiproc.geojson")

In [ ]:
gdf